<a href="https://colab.research.google.com/github/maxwellowusu/movie_recommender_system_ML/blob/main/movie_recommender_max.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movies Recommendation System
## Machine Learning 1
Group Project by Maxwell, Michael, Yaswanth 




### Introduction 

### Motivation

### Research Question 
- Can we predict movie rating ? 

In [ ]:

from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/Colab Notebooks/movie_recommender_project/'

# Get the absolute path of the shallow utilities folder
abspath_util_shallow = '/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/code/utilities/p2_shallow_learning/'

# Get the absolute path of the shallow models folder
abspath_model_shallow = '/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/code/models/p2_shallow_learning/'

Mounted at /content/drive


In [ ]:
# Warning 

import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

In [ ]:
# Matplotlib

import matplotlib.pyplot as plt
%matplotlib inline 

# Set matplotlib sizes
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=20)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('legend', fontsize=20)
plt.rc('figure', titlesize=20)

In [ ]:
# Tensorflow

# The magic below allows us to use tensorflow version 2.x
%tensorflow_version 2.x 
import tensorflow as tf
from tensorflow import keras

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# The random seed
random_seed = 42

# Set random seed in tensorflow
tf.random.set_seed(random_seed)

# Set random seed in numpy
import numpy as np
np.random.seed(random_seed)

## Data preprocessing 

In [ ]:
# Change working directory to the absolute path of the shallow utilities folder
%cd $abspath_util_shallow

# Import the shallow utitilities
%run pmlm_utilities_shallow.ipynb

# Get the name of the target
target = 'Rating'

/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/code/utilities/p2_shallow_learning


## Load data


In [ ]:
# let load .dat data 
import pandas as pd

# Load data
movies=pd.read_csv(abspath_curr+'data/movies.dat', sep='::', encoding='latin-1', header=None, names=['MovieId', 'Title', 'Genres'])
ratings=pd.read_csv(abspath_curr+'data/ratings.dat', sep='::', encoding='latin-1', header=None, names=['UserId', 'MovieId', 'Rating', 'Timestamp'])
users=pd.read_csv(abspath_curr+'data/users.dat', sep='::', encoding='latin-1', header=None, names=['UserId', 'Gender', 'Age', 'Occupation', 'Zip-code'])

In [ ]:
movies.head()

,MovieId,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

,UserId,MovieId,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
users.head()

,UserId,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


Merge datasets
 - First merging movie and ratings data on 'MovieId'
 - Second merged movie+ratings data with users data on 'UserId'

In [ ]:
# Merge data frames
data=pd.merge(left=movies, right=ratings, how='inner', on='MovieId')
data=pd.merge(left=data, right=users, how='inner', on='UserId')
print(data.shape)
data.head()

(1000209, 10)


,MovieId,Title,Genres,UserId,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067


In [ ]:
from sklearn.model_selection import train_test_split

# Divide the training data into training (80%) and testing (20%)
df_train, df_test = train_test_split(data, train_size=0.80, random_state=random_seed)

# Divide the training data into training (80%) and validation (20%)
df_train, df_val = train_test_split(df_train, train_size=0.80, random_state=random_seed)

# Reset the index
df_train, df_val, df_test = df_train.reset_index(drop=True), df_val.reset_index(drop=True), df_test.reset_index(drop=True)

In [ ]:
# print dimension
print("trainset shape:", df_train.shape)
print("validationset shape:", df_val.shape)
print("testset shape:", df_test.shape)

trainset shape: (640133, 10)
validationset shape: (160034, 10)
testset shape: (200042, 10)


## Handling data time variables 

In [ ]:
# Get the date time variables
datetime_vars = []

# Call datetime_transformer on df_train
# See the implementation in pmlm_utilities.ipynb
df_train = datetime_transformer(df_train, datetime_vars)

# Print the first 5 rows of df_train
df_train.head()

# Call datetime_transformer on df_val
# See the implementation in pmlm_utilities.ipynb
df_val = datetime_transformer(df_val, datetime_vars)

# Print the first 5 rows of df_val
df_val.head()

# Call datetime_transformer on df_test
# See the implementation in pmlm_utilities.ipynb
df_test = datetime_transformer(df_test, datetime_vars)

# Print the first 5 rows of df_test
df_test.head()

,MovieId,Title,Genres,UserId,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,588,Aladdin (1992),Animation|Children's|Comedy|Musical,510,5,976207924,M,18,12,55109
1,3203,Dead Calm (1989),Thriller,2279,4,974561199,M,50,7,08816
2,586,Home Alone (1990),Children's|Comedy,678,4,989239492,M,25,0,34952
3,345,"Adventures of Priscilla, Queen of the Desert, ...",Comedy|Drama,733,5,975482948,M,35,16,30034
4,1041,Secrets & Lies (1996),Drama,4771,3,993357573,F,25,4,04101


## Handling missing data

In [ ]:
# Combine df_train, df_val and df_test
df = pd.concat([df_train, df_val, df_test], sort=False)

In [ ]:
# Call nan_checker on df
# See the implementation in pmlm_utilities.ipynb
df_nan = nan_checker(df)

# Print df_nan
df_nan

,var,proportion,dtype


In [ ]:
# Print the unique data type of variables with NaN
pd.DataFrame(df_nan['dtype'].unique(), columns=['dtype'])

,dtype


In [ ]:
# Get the variables with missing values, their proportion of missing values and data type
df_miss = df_nan[df_nan['dtype'] == 'float64'].reset_index(drop=True)

# Print df_miss
df_miss

,var,proportion,dtype


## Encoding data 

In [ ]:
# Combine df_train, df_val and df_test
df = pd.concat([df_train, df_val, df_test], sort=False)

# Print the unique data type of variables in df
pd.DataFrame(df.dtypes.unique(), columns=['dtype'])

,dtype
0,int64
1,object


### Identify categorical data

In [ ]:
# Call cat_var_checker on df
# See the implementation in pmlm_utilities.ipynb
df_cat = cat_var_checker(df)

# Print the dataframe
df_cat

,var,nunique
0,Title,3706
1,Zip-code,3439
2,Genres,301
3,Gender,2


### Encode Genres
Genre (Multilabel Categorical Variable) using countvectorize 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def token_creator(genres):
  l = genres.split('|')
  return l

cv = CountVectorizer(analyzer=token_creator)

matrix_genres = cv.fit_transform(df.Genres)

df_genres=pd.DataFrame(matrix_genres.toarray(), columns=cv.get_feature_names())

In [ ]:
df_genres.head()

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [ ]:
df_movies=pd.concat([data, df_genres], axis=1)
df_movies.drop(['Genres'], axis=1, inplace=True)

In [ ]:
df_movies.head()

,MovieId,Title,UserId,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1,5,978824268,F,1,10,48067,1,...,0,0,0,0,0,0,1,1,0,0
1,48,Pocahontas (1995),1,5,978824351,F,1,10,48067,0,...,0,0,0,0,0,0,0,0,0,0
2,150,Apollo 13 (1995),1,5,978301777,F,1,10,48067,0,...,0,0,0,0,0,0,0,0,0,0
3,260,Star Wars: Episode IV - A New Hope (1977),1,4,978300760,F,1,10,48067,1,...,0,0,0,0,0,0,0,0,0,0
4,527,Schindler's List (1993),1,5,978824195,F,1,10,48067,0,...,0,0,0,0,0,0,1,1,0,0


### Encode Gender
#### Gender (Binary Categorical Variable)
- Female = 0
- Male = 1

In [ ]:
# Encode gender inot 0 and 1
df_movies['Gender']=np.where(df_movies['Gender']=='F',0,1)

## Drop unused columns

In [ ]:
df_movies.drop(['MovieId', 'Title', 'UserId', 'Timestamp', 'Zip-code'], axis=1, inplace=True)

In [ ]:
df_movies.head()

,Rating,Gender,Age,Occupation,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,5,0,1,10,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
1,5,0,1,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,1,10,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,1,10,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,1,10,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


### Separating the training, validation and test data

In [ ]:
# Separating the training data
df_train = df_movies.iloc[:df_train.shape[0], :]

# Separating the validation data
df_val = df_movies.iloc[df_train.shape[0]:df_train.shape[0] + df_val.shape[0], :]

# Separating the test data
df_test = df_movies.iloc[df_train.shape[0] + df_val.shape[0]:, :]

In [ ]:
df_train.head()

,Rating,Gender,Age,Occupation,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,5,0,1,10,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
1,5,0,1,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,1,10,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,1,10,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,1,10,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


## Split features and target

In [ ]:
# Get the feature matrix
X_train = df_train[np.setdiff1d(df_train.columns, [target])].values
X_val = df_val[np.setdiff1d(df_val.columns, [target])].values
X_test = df_test[np.setdiff1d(df_test.columns, [target])].values

# Get the target vector
y_train = df_train[target].values
y_val = df_val[target].values
y_test = df_test[target].values

## Scaling the data
### Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# The MinMaxScaler
mms = MinMaxScaler()

In [ ]:
# Normalize the training data
X_train = mms.fit_transform(X_train)

# Normalize the validation data
X_val = mms.transform(X_val)

# Normalize the test data
X_test = mms.transform(X_test)

## Hyperparameter Tuning
### Creating the dictionary of the models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

models = {'lr': LogisticRegression(class_weight='balanced', random_state=random_seed),
          'mlpc': MLPClassifier(early_stopping=True, random_state=random_seed),
          'rfc': RandomForestClassifier(class_weight='balanced', random_state=random_seed),
          'hgbc': HistGradientBoostingClassifier(random_state=random_seed)}

### Creating the dictionary of the pipelines

In [ ]:
from sklearn.pipeline import Pipeline

pipes = {}

for acronym, model in models.items():
    pipes[acronym] = Pipeline([('model', model)])

### Getting the predefined split cross-validator

In [ ]:
X_train_val, y_train_val, ps = get_train_val_ps(X_train, y_train, X_val, y_val)

### GridSearchCV
Creating the dictionary of the parameter grids

In [ ]:
param_grids = {}

### Parameter grid for LogisticRegression

In [ ]:
# The parameter grid of tol
tol_grid = [10 ** -7, 10 ** -5, 10 ** -4, 10 ** -3]

# The parameter grid of C
C_grid = [0.1, 1, 11, 13]

# Update param_grids
param_grids['lr'] = [{'model__tol': tol_grid,
                      'model__C': C_grid}]

### Parameter grid for MLPClassifier

In [ ]:
# The grids for alpha
alpha_grids = [10 ** i for i in range(-5, -3)]

# The grids for learning_rate_init
learning_rate_init_grids = [10 ** i for i in range(-3, 2)]

# Update param_grids
param_grids['mlpc'] = [{'model__alpha': alpha_grids,
                        'model__learning_rate_init': learning_rate_init_grids}]

### Parameter grid for Random Forest 

In [ ]:
# The grids for min_samples_split
min_samples_split_grids = [21, 23, 30, 27, 30]

# The grids for min_samples_leaf
min_samples_leaf_grids = [1, 3, 7, 9, 11]

# Update param_grids
param_grids['rfc'] = [{'model__min_samples_split': min_samples_split_grids,
                       'model__min_samples_leaf': min_samples_leaf_grids}]

###  Parameter grid for histogram-based gradient boosting

In [ ]:
# The grids for learning_rate
learning_rate_grids = [10 ** i for i in range(-3, 2)]

# The grids for min_samples_leaf
min_samples_leaf_grids = [1, 20, 27]

# Update param_grids
param_grids['hgbc'] = [{'model__learning_rate': learning_rate_grids,
                        'model__min_samples_leaf': min_samples_leaf_grids}]

### Creating the directory for the cv results produced by GridSearchCV

In [ ]:
# Make directory
import os
directory = os.path.dirname(abspath_curr + '/result/cv_results/GridSearchCV/')
if not os.path.exists(directory):
    os.makedirs(directory)

### Tuning the hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV

# The list of [best_score_, best_params_, best_estimator_] obtained by GridSearchCV
best_score_params_estimator_gs = []

# For each model
for acronym in pipes.keys():
    # GridSearchCV
    gs = GridSearchCV(estimator=pipes[acronym],
                      param_grid=param_grids[acronym],
                      scoring='f1_macro',
                      n_jobs=2,
                      cv=ps,
                      return_train_score=True)
        
    # Fit the pipeline
    gs = gs.fit(X_train_val, y_train_val)
    
    # Update best_score_params_estimator_gs
    best_score_params_estimator_gs.append([gs.best_score_, gs.best_params_, gs.best_estimator_])
    
    # Sort cv_results in ascending order of 'rank_test_score' and 'std_test_score'
    cv_results = pd.DataFrame.from_dict(gs.cv_results_).sort_values(by=['rank_test_score', 'std_test_score'])
    
    # Get the important columns in cv_results
    important_columns = ['rank_test_score',
                         'mean_test_score', 
                         'std_test_score', 
                         'mean_train_score', 
                         'std_train_score',
                         'mean_fit_time', 
                         'std_fit_time',                        
                         'mean_score_time', 
                         'std_score_time']
    
    # Move the important columns ahead
    cv_results = cv_results[important_columns + sorted(list(set(cv_results.columns) - set(important_columns)))]

    # Write cv_results file
    cv_results.to_csv(path_or_buf=abspath_curr + '/result/cv_results/GridSearchCV/' + acronym + '.csv', index=False)

# Sort best_score_params_estimator_gs in descending order of the best_score_
best_score_params_estimator_gs = sorted(best_score_params_estimator_gs, key=lambda x : x[0], reverse=True)

# Print best_score_params_estimator_gs
pd.DataFrame(best_score_params_estimator_gs, columns=['best_score', 'best_param', 'best_estimator'])

,best_score,best_param,best_estimator
0,0.185309,"{'model__min_samples_leaf': 1, 'model__min_sam...","((DecisionTreeClassifier(max_features='auto', ..."
1,0.176156,"{'model__C': 0.1, 'model__tol': 1e-07}","(LogisticRegression(C=0.1, class_weight='balan..."
2,0.141318,"{'model__learning_rate': 10, 'model__min_sampl...",(HistGradientBoostingClassifier(learning_rate=...
3,0.106335,"{'model__alpha': 0.0001, 'model__learning_rate...","(MLPClassifier(early_stopping=True, random_sta..."


## Model section 

In [ ]:
# Make directory
directory = os.path.dirname(abspath_curr + '/result/boo/submission/')
if not os.path.exists(directory):
    os.makedirs(directory)

## Generating the submission file

In [51]:
# Get the best_score, best_param and best_estimator obtained by GridSearchCV
best_score_gs, best_params_gs, best_estimator_gs = best_score_params_estimator_gs[0]

# Get the prediction on the testing data using best_model
y_test_pred = best_estimator_gs.predict(X_test)

# # Transform y_test_pred back to the original class
# y_test_pred = le.inverse_transform(y_test_pred)

# # Get the submission dataframe
# df_submit = pd.DataFrame(np.hstack((df_raw_test['id'].to_numpy().reshape(-1, 1), y_test_pred.reshape(-1, 1))),
#                          columns=['id', target])                                                                                      

# Generate the submission file

### Test set accuracy

In [52]:
from sklearn.metrics import classification_report, confusion_matrix


print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_test_pred))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_test_pred))

=== Confusion Matrix ===
[[ 2916  1742  1460  1864  1799]
 [ 4922  3248  2723  3589  3326]
 [11051  7975  7621  9516  9160]
 [17334 12041 11749 14857 14541]
 [14354  9515  9266 11636 11837]]


=== Classification Report ===
              precision    recall  f1-score   support

           1       0.06      0.30      0.10      9781
           2       0.09      0.18      0.12     17808
           3       0.23      0.17      0.20     45323
           4       0.36      0.21      0.27     70522
           5       0.29      0.21      0.24     56608

    accuracy                           0.20    200042
   macro avg       0.21      0.21      0.18    200042
weighted avg       0.27      0.20      0.22    200042

